# Fine-tune Gemma-2-2b for Code Audit (Kaggle T4)

**Production Notebook** | 2026 | ~25 min training

**Workflow:**
1. Run Cell 1 (Install) → RESTART KERNEL
2. Run Cells 2-7 (skip Cell 1)

## 1. Install (then RESTART KERNEL!)

In [ ]:
# KAGGLE UNSLOTH STABLE INSTALL (2026)
# Based on working config from qwen3-phone-deploy.ipynb

import subprocess
import sys

def run_pip(cmd, msg):
    print(msg)
    # Using --no-cache-dir to ensure fresh binaries
    full_cmd = f"pip install -q --no-cache-dir {cmd}"
    subprocess.run(full_cmd, shell=True, capture_output=True)

print("="*60)
print("🚀 KAGGLE UNSLOTH STABLE INSTALL")
print("="*60)

print("[1/5] Cleaning environment...")
subprocess.run("pip uninstall -y torchao unsloth unsloth_zoo transformers -q", shell=True)

run_pip(
    "fsspec==2024.9.0 datasets==4.2.0 huggingface_hub>=0.23.0", 
    "[2/5] Installing base deps (fsspec 2024.9.0, datasets 4.2.0)..."
)

run_pip(
    "peft accelerate bitsandbytes trl transformers>=4.45.0",
    "[3/5] Installing training components (transformers >= 4.45.0)..."
)

run_pip(
    '"unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"',
    "[4/5] Installing Unsloth (GitHub Latest)..."
)

run_pip("xformers==0.0.28.post3", "[5/5] Installing xformers...")

print("\n" + "="*60)
print("✅ INSTALLATION COMPLETE!")
print("="*60)
print("\n" + "*"*60)
print("*** IMPORTANT: RESTART KERNEL NOW! ***")
print("*** Then run the cells below in order. ***")
print("*"*60)


## 2. Load Model

**Crucial:** Import `unsloth` before anything else!

In [ ]:
import unsloth  # Must be first!
import torch
from unsloth import FastLanguageModel

print(f"✅ Unsloth: {unsloth.__version__}")
print(f"✅ Torch: {torch.__version__}")

# Configuration
MODEL_NAME = "unsloth/gemma-2-2b-it-bnb-4bit"
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=LOAD_IN_4BIT,
)

## 3. Load Model

In [ ]:
MODEL_NAME = "unsloth/gemma-2-2b-it-bnb-4bit"
MAX_SEQ_LENGTH = 2048

print(f"[INFO] Loading {MODEL_NAME}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

print(f"\n[OK] Model loaded!")
print(f"[OK] VRAM: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## 4. Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("[OK] LoRA added!")
model.print_trainable_parameters()

## 5. Load Dataset

In [ ]:
from datasets import load_dataset
import glob

# Find dataset
for p in ["/kaggle/input/audit-dataset/audit_dataset.jsonl",
          "/kaggle/input/*/audit_dataset.jsonl",
          "/kaggle/input/*/*.jsonl"]:
    m = glob.glob(p)
    if m:
        dataset_path = m[0]
        break
else:
    raise FileNotFoundError("Upload audit_dataset.jsonl!")

print(f"[INFO] Loading: {dataset_path}")
dataset = load_dataset("json", data_files=dataset_path, split="train")

ALPACA = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS = tokenizer.eos_token

def fmt(ex):
    return {"text": [ALPACA.format(i, o) + EOS for i, o in zip(ex["instruction"], ex["output"])]}

dataset = dataset.map(fmt, batched=True)
print(f"[OK] {len(dataset)} examples")

## 6. Train (~25 min)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        output_dir="./outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)

print("[INFO] Training...")
stats = trainer.train()
print(f"\n[OK] Done in {stats.metrics['train_runtime']:.0f}s")

## 7. Save Model

In [ ]:
# Test
FastLanguageModel.for_inference(model)
test = "Analyze test coverage: 330 files, 5 executable, 0% coverage"
inputs = tokenizer([ALPACA.format(test, "")], return_tensors="pt").to("cuda")
out = model.generate(**inputs, max_new_tokens=256, use_cache=True)
print("[TEST]\n", tokenizer.decode(out[0]))

# Save
model.save_pretrained("audit-gemma-v1")
tokenizer.save_pretrained("audit-gemma-v1")
print("\n[OK] Saved to audit-gemma-v1/")